In [1]:
import csv
import habitat_sim
import math
import magnum as mn
import numpy as np
import operator
import os

In [7]:
def read_scene(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        title = next(reader)[13:-1]

        scene = []
        for row in reader:
            count_list = row[13:-1]

            indices = [(int(item), index) for index, item in enumerate(count_list)]
            sorted_list = sorted(indices, key=lambda x: x[0], reverse=True)
            max_indices = [pair[1] for pair in sorted_list[:1]]
            
            scene.append(row[0] + '_' + '_'.join([title[i] for i in max_indices]))

    return scene


In [8]:
def make_setting(scene_file):
    # test_scene = "./scene_datasets/hssd-hab/scenes/102343992.scene_instance.json"  
    # test_scene = "102343992"
    # scene_dataset = "/data2/liangxiwen/habitat-lab/habitat_Nav_Grasp/data/scene_datasets/hssd-hab/hssd-hab.scene_dataset_config.json"
    if int(scene_file[2]) < 8:
        split = 'train/'
    else:
        split = 'val/'
    test_scene = '/data2/liangxiwen/habitat-lab/data/scene_datasets/hm3d/' + split + scene_file
    scene_dataset = '/data2/liangxiwen/habitat-lab/data/scene_datasets/hm3d/hm3d_annotated_basis.scene_dataset_config.json'

    rgb_sensor = True  # @param {type:"boolean"}
    depth_sensor = True  # @param {type:"boolean"}
    semantic_sensor = True  # @param {type:"boolean"}

    height = 1
    sim_settings = {
        "width": 512,  # Spatial resolution of the observations
        "height": 512,
        "scene": test_scene,  # Scene path
        "scene_dataset": scene_dataset,
        "default_agent": 0,
        "sensor_height": height,  # Height of sensors in meters
        "color_sensor_f": rgb_sensor,  # RGB sensor
        "color_sensor_l": rgb_sensor,  # RGB sensor
        "color_sensor_r": rgb_sensor,  # RGB sensor
        "depth_sensor": depth_sensor,  # Depth sensor
        "semantic_sensor": semantic_sensor,  # Semantic sensor
        "seed": 1,  # used in the random navigation
        "enable_physics": False,  # kinematics only
    }
    return sim_settings

def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_id = settings["scene"]
    sim_cfg.enable_physics = settings["enable_physics"]

    if "scene_dataset" in settings:
        sim_cfg.scene_dataset_config_file = settings["scene_dataset"]

    # Note: all sensors must have the same resolution
    sensors = {
        "color_sensor_f": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "orientation": [0.0, 0.0, 0.0],
        },
        "color_sensor_l": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "orientation": [0.0, math.pi / 3.0, 0.0],
        },
        "color_sensor_r": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "orientation": [0.0, -math.pi / 3.0, 0.0],
        },
        "depth_sensor": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "orientation": [0.0, 0.0, 0.0],
        },
        "semantic_sensor": {
            "sensor_type": habitat_sim.SensorType.SEMANTIC,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
            "orientation": [0.0, 0.0, 0.0],
        },
    }
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.CameraSensorSpec()
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]
            sensor_spec.orientation = sensor_params["orientation"]
            sensor_specs.append(sensor_spec)
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
        "stop": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0)
        ),
    }
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

def print_scene_recur(sim, file):
    out_path = "/data2/liangxiwen/habitat-lab/nav_gen/data/gen_data/scene/" + file
    useless = ["wall", "frame", "floor", "sheet", "Unknown", "stairs", "unknown",
                "ceiling", "window", "curtain", "pillow", "beam", "decoration"]
    # if not os.path.exists(out_path):
    #     os.makedirs(out_path)
    scene = sim.semantic_scene
    print(
        f"House has {len(scene.levels)} levels, {len(scene.regions)} regions and {len(scene.objects)} objects"
    )
    print(f"House center:{scene.aabb.center} dims:{scene.aabb.sizes}")
    for region in scene.regions:
        print(
            f"Region id:{region.id},"
            f" center:{region.aabb.center}, dims:{region.aabb.sizes}"
        )
        with open(out_path + ".txt",'a') as f:
            f.write(
                f"Region id:{region.id},"
                f" position:{region.aabb.center}"
                f"\n"
            )
        for obj in region.objects:
            obj_id = obj.id.split("_")[1]
            print(
                f"Object id:{obj_id}, category:{obj.category.name()},"
                f" center:{obj.aabb.center}, dims:{obj.aabb.sizes}"
            )
            if any(c in obj.category.name() for c in useless):
                continue
            else:
                with open(out_path + ".txt",'a') as f:
                    f.write(
                        f"Id:{obj_id}, name:{obj.category.name()},"
                        f" position:{[obj.aabb.center[0], obj.aabb.center[1], obj.aabb.center[2]]}"
                        f"\n"
                    )

In [9]:
file = '/data2/liangxiwen/habitat-lab/nav_gen/data/gen_data/scene/Per_Scene_Total_Weighted_Votes.csv'
scene = read_scene(file)
print(scene)

['00006-HkseAnWCgqk_Bathroom', '00009-vLpv2VX547B_Bedroom', '00016-qk9eeNeR4vw_Bathroom', '00017-oEPjPNSPmzL_Kitchen', '00020-XYyR54sxe6b_Bathroom', '00022-gmuS7Wgsbrx_Bathroom', '00023-zepmXAdrpjR_Bathroom', '00025-ixTj1aTMup2_Bathroom', '00031-Wo6kuutE9i7_Bathroom', '00033-oPj9qMxrDEa_Kitchen', '00034-6imZUJGRUq4_Bedroom', '00035-3XYAD64HpDr_Bathroom', '00043-Jfyvj3xn2aJ_Bathroom', '00055-HxmXPBbFCkH_Bedroom', '00057-1UnKg1rAb8A_Bedroom', '00059-kJxT5qssH4H_Bathroom', '00062-ACZZiU6BXLz_Bathroom', '00064-gQgtJ9Stk5s_Bathroom', '00081-5biL7VEkByM_Bathroom', '00087-YY8rqV6L6rf_Bathroom', '00096-6HRFAUDqpTb_Bathroom', '00099-226REUyJh2K_Bathroom', '00105-xWvSkKiWQpC_Office', '00108-oStKKWkQ1id_Bathroom', '00109-GTV2Y73Sn5t_Bathroom', '00135-HeSYRw7eMtG_Bathroom', '00141-iigzG1rtanx_Bathroom', '00143-5Kw4nGdqYtS_Bathroom', '00149-UuwwmrTsfBN_Bathroom', '00150-LcAd9dhvVwh_Bathroom', '00155-iLDo95ZbDJq_Bathroom', '00164-XfUxBGTFQQb_Bathroom', '00166-RaYrxWt5pR1_Bathroom', '00168-bHKTDQFJxT

In [10]:
for i in range(len(scene)):
    l = scene[i].split('_')
    print(l)
    file = l[0]

    sim_settings = make_setting(file)
    cfg = make_cfg(sim_settings)
    sim = habitat_sim.Simulator(cfg)

    print_scene_recur(sim, file)
    sim.close()

['00006-HkseAnWCgqk', 'Bathroom']
House has 0 levels, 12 regions and 453 objects
House center:[0. 0. 0.] dims:[-inf -inf -inf]
Region id:_-1, center:[0. 0. 0.], dims:[-inf -inf -inf]
Object id:0, category:Unknown, center:[-5.6580954   2.8323302  -0.16315174], dims:[2.6479397 0.6200037 9.943148 ]
Region id:_0, center:[0. 0. 0.], dims:[-inf -inf -inf]
Object id:1, category:floor, center:[-4.93333     0.00873863 -3.5729375 ], dims:[5.3865194  0.06718805 2.9330044 ]
Object id:2, category:wall, center:[-4.9734855  1.2382828 -2.0966215], dims:[5.3371572  2.5078962  0.13999939]
Object id:3, category:wall, center:[-7.059587   1.2477721 -4.9752336], dims:[1.089654   2.5074313  0.11437988]
Object id:4, category:wall, center:[-2.6517339  1.2482901 -4.9236526], dims:[0.7402458  2.4919052  0.08351994]
Object id:5, category:wall, center:[-2.3052154  1.2463142 -3.4907212], dims:[0.07921314 2.4879754  2.848158  ]
Object id:6, category:wall, center:[-4.7618027  1.2436011 -4.983102 ], dims:[3.4796736  2

KeyboardInterrupt: 